In [17]:
using CUDA
using ProgressMeter
using DelimitedFiles
include("../src/forces.jl")
include("../src/neighbor.jl")

cu_knn (generic function with 1 method)

In [18]:
# Running fusion of the two aggregates
# Physical Conditions
R_agg = 4

# Time model Conditions
T_f = 100000
dt = 0.1

# Neighbor Conditions
n_knn = 50
nn = 12

# Forces Conditions
r_max = 2.6
fp = 0.01
K = 1.0
s = 2.0

# Coding C0nditions
n_text = 50
num_agg = 2

2

In [19]:
X = Float32.(readdlm("../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

46×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.5  -2.89  -2.45
 -2.5  -1.15  -2.45
 -0.5  -1.15  -2.45
  1.5  -1.15  -2.45
 -1.5   0.58  -2.45
  0.5   0.58  -2.45
  2.5   0.58  -2.45
 -0.5   2.31  -2.45
  1.5   2.31  -2.45
 -2.5  -2.31  -0.82
 -0.5  -2.31  -0.82
  1.5  -2.31  -0.82
 -3.5  -0.58  -0.82
  ⋮           
 -2.5   2.31   0.82
 -0.5   2.31   0.82
  1.5   2.31   0.82
 -0.5  -2.31   2.45
  1.5  -2.31   2.45
 -1.5  -0.58   2.45
  0.5  -0.58   2.45
  2.5  -0.58   2.45
 -2.5   1.15   2.45
 -0.5   1.15   2.45
  1.5   1.15   2.45
  0.5   2.89   2.45

In [20]:
cu_knn()
idx

12×46 Matrix{CartesianIndex{2}}:
 CartesianIndex(1, 1)   CartesianIndex(2, 2)   …  CartesianIndex(46, 46)
 CartesianIndex(11, 1)  CartesianIndex(13, 2)     CartesianIndex(36, 46)
 CartesianIndex(12, 1)  CartesianIndex(14, 2)     CartesianIndex(37, 46)
 CartesianIndex(3, 1)   CartesianIndex(5, 2)      CartesianIndex(44, 46)
 CartesianIndex(4, 1)   CartesianIndex(3, 2)      CartesianIndex(45, 46)
 CartesianIndex(15, 1)  CartesianIndex(10, 2)  …  CartesianIndex(33, 46)
 CartesianIndex(25, 1)  CartesianIndex(17, 2)     CartesianIndex(22, 46)
 CartesianIndex(14, 1)  CartesianIndex(11, 2)     CartesianIndex(32, 46)
 CartesianIndex(16, 1)  CartesianIndex(27, 2)     CartesianIndex(34, 46)
 CartesianIndex(10, 1)  CartesianIndex(18, 2)     CartesianIndex(35, 46)
 CartesianIndex(2, 1)   CartesianIndex(15, 2)  …  CartesianIndex(43, 46)
 CartesianIndex(6, 1)   CartesianIndex(6, 2)      CartesianIndex(41, 46)

In [21]:
r = reshape(repeat(X, inner=(nn,1)), nn, size(X)[1], 3) - X[getindex.(idx,1),:]

12×46×3 CuArray{Float32, 3, CUDA.Mem.DeviceBuffer}:
[:, :, 1] =
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  1.0   1.0   1.0   1.0   1.0   1.0      1.0   1.0   1.0   1.0   1.0   1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0     -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
  1.0  -1.0   1.0   1.0   1.0   1.0      1.0   1.0  -1.0   1.0   1.0   1.0
 -1.0  -2.0  -1.0  -1.0  -1.0  -1.0     -1.0   1.0  -2.0  -1.0  -1.0  -1.0
  0.0   0.0   2.0   2.0  -1.0   1.0  …   1.0   2.0   0.0   2.0   2.0   0.0
  0.0   0.0  -2.0   0.0  -2.0  -1.0     -1.0   0.0   0.0  -2.0   0.0   0.0
  2.0  -2.0   0.0   1.0   0.0   2.0      2.0   0.0  -2.0   0.0   1.0   2.0
 -2.0   0.0  -1.0   0.0   0.0  -2.0     -2.0   2.0   0.0  -1.0   0.0  -2.0
  3.0  -2.0   0.0   2.0   2.0   0.0      0.0   0.0  -2.0   0.0   2.0   3.0
  3.0  -3.0   2.0   0.0  -2.0   0.0  …   0.0   3.0  -3.0   2.0   0.0   3.0
  0.0  -3.0  -2.0   2.0   0.0   2.0      2.0   3.0  -3.0  -2.0   2.0   0.0

[:, :, 2] =
  0.0    0.0    0.0    

In [22]:
dist = ((sum(r .^ 2, dims=3)) .^ 0.5)[:,:,1]
dist = reshape(repeat((dist), outer=(1,3)) ,nn ,size(X)[1], 3)

12×46×3 CuArray{Float64, 3, CUDA.Mem.DeviceBuffer}:
[:, :, 1] =
 0.0      0.0      0.0      0.0      …  0.0      0.0      0.0      0.0
 1.99832  1.99544  1.99544  1.99544     1.99544  1.99544  1.99544  1.99832
 1.99832  1.99544  1.99544  1.99544     1.99544  1.99544  1.99544  1.99832
 2.00689  1.99822  1.99822  1.99822     1.99822  1.99822  1.99822  2.00689
 2.00689  2.0      1.99822  1.99822     2.0      1.99822  1.99822  2.00689
 2.82719  2.00063  2.0      2.0      …  2.00063  2.0      2.0      2.82719
 3.27     2.81902  2.0      2.00063     2.81902  2.0      2.00063  3.27
 3.46309  2.82887  2.00063  2.00689     2.82887  2.00063  2.00689  3.46309
 3.46309  3.27     2.00689  2.81902     3.27     2.00689  2.81902  3.46309
 3.46313  3.45643  2.81902  2.82887     3.45643  2.81902  2.82887  3.46313
 3.46808  3.46147  2.82887  3.27     …  3.46147  2.82887  3.27     3.46808
 3.47     3.46308  2.82887  3.45643     3.46308  2.82887  3.45643  3.47

[:, :, 2] =
 0.0      0.0      0.0      0.0  

In [23]:
# Finding forces for each cell
F = (- K.*((dist .- r_max).^2) .* (dist .- s)) .* r ./ dist
# Deleting Forces greater than R_Max
F[dist .>  r_max] .= 0
# F[isnan.(F)] .= 0
F

12×46×3 CuArray{Float64, 3, CUDA.Mem.DeviceBuffer}:
[:, :, 1] =
 NaN            NaN            NaN            …  NaN            NaN
   0.000303548    0.000834305    0.000834305       0.000834305    0.000303548
  -0.000303548   -0.000834305   -0.000834305      -0.000834305   -0.000303548
  -0.00120741    -0.000321816    0.000321816       0.000321816   -0.00120741
   0.00120741     0.0           -0.000321816      -0.000321816    0.00120741
   0.0           -0.0           -0.0          …   -0.0            0.0
   0.0            0.0            0.0              -0.0            0.0
   0.0            0.0           -0.0              -0.00120741     0.0
   0.0            0.0            0.00120741        0.0            0.0
   0.0            0.0            0.0               0.0            0.0
   0.0            0.0            0.0          …    0.0            0.0
   0.0            0.0            0.0               0.0            0.0

[:, :, 2] =
 NaN            NaN            NaN            …  NaN   

In [24]:
# Calculating de dX   -> dX[i,:] +=  r/dist * F
dX = sum(F[2:end,:,:]; dims=1)[1,:,:]

46×3 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
  0.0           0.00384969   -0.000989566
 -0.000321816  -0.00163805   -0.00253689
  0.00120741   -0.00429569   -0.00253689
 -0.00120741   -0.00429569   -0.00253689
 -0.000321816  -0.00052456   -0.00253689
  0.0          -0.0010813    -0.00253689
  0.000643633  -0.0010813    -0.00253689
  0.0           0.000631174  -0.00080662
  0.0           0.000631174  -0.00080662
  0.00113928    0.000128299   0.00419123
 -0.000303548  -0.000356425   0.00655443
  0.000303548  -0.000356425   0.00655443
 -0.000835033   0.00188147    0.00492005
  ⋮                          
  0.00113928   -0.000128299  -0.00419123
 -0.000303548   0.000356425  -0.00655443
  0.000303548   0.000356425  -0.00655443
  0.0          -0.000631174   0.00080662
  0.0          -0.000631174   0.00080662
 -0.000321816   0.00052456    0.00253689
  0.0           0.0010813     0.00253689
  0.000643633   0.0010813     0.00253689
 -0.000321816   0.00163805    0.00253689
  0.00120741    0.

In [26]:
# Adding fp for random forces
t=0
rand_idx = getindex.(idx, 1)[getindex.(rand(2:nn,2*size(X,1)) ,1),:]

92×46 Matrix{Int64}:
  4   3   6   7   3   4   9  22  23  …  25  26  43  39  45  44  41  42  45
  6   6  12  18  32  14   3   7   5     42  40  14  32  38  41  37  28  41
 14  11  11   1  14   5  23  17  18     27  28  32  40  26  36  36  46  32
  4   3   6   7   3   4   9  22  23     25  26  43  39  45  44  41  42  45
 25  17   4  12   6   9  16  18  19     28  29  41  45  34  27  45  37  22
 10  18  18  11  13  15  34  36  37  …  11  12  31  33  16  28  28  36  35
 11  13  14  15  17  18  19   5   6     40  41  27  28  29  31  32  33  36
  6   6  12  18  32  14   3   7   5     42  40  14  32  38  41  37  28  41
 25  17   4  12   6   9  16  18  19     28  29  41  45  34  27  45  37  22
  3   5   5   6   2   3   4  21  22     24  25  38  38  39  40  40  41  44
  6   6  12  18  32  14   3   7   5  …  42  40  14  32  38  41  37  28  41
 14  11  11   1  14   5  23  17  18     27  28  32  40  26  36  36  46  32
  6   6  12  18  32  14   3   7   5     42  40  14  32  38  41  37  28  41
  ⋮ 

In [27]:
r_p = X - X[rand_idx[mod(t, size(X,1))+1, :], :]

46×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.0  -1.74   0.0
 -2.0   0.0    0.0
 -1.0  -1.73   0.0
 -1.0  -1.73   0.0
 -1.0   1.73   0.0
 -1.0   1.73   0.0
  1.0  -1.73   0.0
 -1.0  -0.58  -1.63
 -1.0  -0.58  -1.63
  0.0  -1.16   1.63
  2.0   0.0    0.0
  2.0   0.0    0.0
 -2.0   0.0    0.0
  ⋮           
  0.0   1.16  -1.63
  2.0   0.0    0.0
  2.0   0.0    0.0
 -1.0   0.58   1.63
 -1.0   0.58   1.63
  1.0  -1.73   0.0
 -1.0   1.73   0.0
  1.0  -1.73   0.0
 -2.0   0.0    0.0
 -1.0   1.73   0.0
 -1.0   1.73   0.0
 -1.0   1.74   0.0

In [28]:
dist_p = (sum(r_p .^ 2, dims=2).^ 0.5)

46×1 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
 2.0068882102377086
 2.0
 1.9982242449567402
 1.9982242449567402
 1.9982242449567402
 1.9982242449567402
 1.9982242449567402
 1.9983244075793216
 1.9983244075793216
 2.000625035846952
 2.0
 2.0
 2.0
 ⋮
 2.000625035846952
 2.0
 2.0
 1.9983244075793216
 1.9983244075793216
 1.9982242449567402
 1.9982242449567402
 1.9982242449567402
 2.0
 1.9982242449567402
 1.9982242449567402
 2.0068882102377086

In [29]:
dX = dX - fp .* (r_p ./ dist_p)

46×3 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
  0.00498284   0.0125198    -0.000989566
  0.00967818  -0.00163805   -0.00253689
  0.00621186   0.004362     -0.00253689
  0.00379703   0.004362     -0.00253689
  0.00468263  -0.00918225   -0.00253689
  0.00500444  -0.00973899   -0.00253689
 -0.00436081   0.00757638   -0.00253689
  0.00500419   0.00353361    0.00735021
  0.00500419   0.00353361    0.00735021
  0.00113928   0.00592649   -0.00395622
 -0.0103035   -0.000356425   0.00655443
 -0.00969645  -0.000356425   0.00655443
  0.00916497   0.00188147    0.00492005
  ⋮                         
  0.00113928  -0.00592649    0.00395622
 -0.0103035    0.000356425  -0.00655443
 -0.00969645   0.000356425  -0.00655443
  0.00500419  -0.00353361   -0.00735021
  0.00500419  -0.00353361   -0.00735021
 -0.00532626   0.00918225    0.00253689
  0.00500444  -0.00757638    0.00253689
 -0.00436081   0.00973899    0.00253689
  0.00967818   0.00163805    0.00253689
  0.00621186  -0.004362      0.00253689
 